In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime as dt

In [3]:
today_str = dt.date.today().strftime("%Y_%m_%d")

# 1. [Stay at home data](https://github.com/COVID19StatePolicy/SocialDistancing): University of Washington

## 1.1 Stay at home data

In [4]:
url = "https://raw.githubusercontent.com/COVID19StatePolicy/SocialDistancing/master/data/USstatesCov19distancingpolicy.csv"
df_stayhome = pd.read_csv(url, encoding = "ISO-8859-1")
df_stayhome.head(1)

,ï»¿location_id,StateFIPS,StatePostal,StateName,StatePolicy,Mandate,StateWide,Curfew,CurfewStart,CurfewEnd,...,DateExpiry,DateEased,DateEnded,DateReexpanded1,DateReeased1,PolicyCodingNotes,PolicySource,LastUpdated,LastUpdatedNotes,ReReviewed
0,523,1,AL,Alabama,EmergDec,1,1,0,NaN,NaN,...,20210706.0,NaN,20210706.0,NaN,NaN,"Public Health Emergency declared, in effect fo...",https://governor.alabama.gov/newsroom/2020/03/...,20210601,Added DateEnded and Updated PolicyCodingNotes,1.0


In [5]:
df_stayhome.StatePolicy.value_counts()

GathRestrict            482
OtherBusinessClose      436
BarRestrict             237
RestaurantRestrict      228
PublicMask              162
StayAtHome              117
SchoolClose             109
BusinessMask             79
Quarantine               60
EmergDec                 57
SchoolMask               46
NEBusinessClose          36
CaseIsolation            19
TravelRestrictEntry       9
TravelRestrictIntra       7
TravelRestrictExit        4
TravelRestrictIntra       1
TravelRestrictEntry       1
Name: StatePolicy, dtype: int64

### 1.1.1 Count types of policies by state

In [7]:
policies = ['OtherBusinessClose','BarRestrict','RestaurantRestrict', 'StayAtHome']
df_policies = pd.DataFrame()
for policy in policies:
    mask = (df_stayhome.StatePolicy==policy) & (df_stayhome.StateWide==1)
    df_temp = df_stayhome.loc[mask, ['StateName','StateWide']].groupby('StateName')['StateWide'].count()
    df_policies = pd.concat([df_policies, df_temp], axis=1)
df_policies.columns = policies
df_policies = df_policies.fillna(0)

In [8]:
df_policies = df_policies.reset_index().rename(columns={'index':'state'})
df_policies.head(2)

,state,OtherBusinessClose,BarRestrict,RestaurantRestrict,StayAtHome
0,Alabama,5.0,2,2,2.0
1,Alaska,7.0,2,2,1.0


# 2. [Cases](https://github.com/nytimes/covid-19-data): NYT

In [9]:
url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv'
df_cases_nyt = pd.read_csv(url, parse_dates=True, index_col='date' )
df_cases_nyt.drop('fips', inplace=True, axis=1) 
df_cases_nyt.head(3)

,state,cases,deaths
date,,,
2020-01-21,Washington,1,0
2020-01-22,Washington,1,0
2020-01-23,Washington,1,0


In [10]:
df_cases_nyt = df_cases_nyt.groupby('state')['cases','deaths'].sum()
df_cases_nyt.head(2)

<ipython-input-10-843fa52f7be8>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_cases_nyt = df_cases_nyt.groupby('state')['cases','deaths'].sum()


,cases,deaths
state,,
Alabama,113683166,2098533
Alaska,12876504,61506
